In [5]:
import requests
from requests.auth import HTTPBasicAuth
import json
import os
from typing import List, Dict, Optional
import time
import re
import uuid

BASE_URL = "http://34.236.171.80"

In [6]:
def login(email: str, password: str):
    """
    Login a user with the given email and password.
    If successful, this function returns an access token.
    """
    url = f"{BASE_URL}/api/user/email-login"
    response = requests.get(url, auth=HTTPBasicAuth(email, password))
    result = response.json()
    token = result.get("data", {}).get("access_token")
    return token

def list_models(
    name: str, token: str, domain=None, username=None, type=None, sub_type=None, access_level=None
):
    """
    List models based on filters for authenticated users.
    Returns:
    - List of models that match the provided filters.
    """
    url = f"{BASE_URL}/api/model/list"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "name": name,
        "domain": domain,
        "username": username,
        "type": type,
        "sub_type": sub_type,
        "access_level": access_level,
    }
    response = requests.get(
        url, headers=headers, params={k: v for k, v in params.items() if v is not None}
    )
    return response.json()

In [2]:
def query_retrieval_model(model_id: str, query: str, token: str):
    """
    Retrieves top k most relevant references to the query from the deployed model.
    Parameters:
    - model_id: <username>/<modelname>
    - query: The query to search for.
    - token: Authorization token from login
    """
    # Define the URL for querying the deployed model
    headers = {"Authorization": f"Bearer {token}"}
    query_url = f"{BASE_URL}/{model_id}/predict"
    # Set up the query parameters
    base_params = {"query": query, "top_k": 5}
    ndb_params = {"constraints": {}}
    # Make a POST request to query the model
    response = requests.post(
        query_url,
        json={"base_params": base_params, "ndb_params": ndb_params},
        headers=headers,
    )
    # Check if the query was successful; if not, raise an exception
    if response.status_code != 200:
        raise Exception(f"Query failed: {response.status_code}, {response.text}")
    return response.json()["data"]["references"]

def query_sentiment_model(model_id: str, query: str, token: str):
    """
    Retrieves top k most relevant references to the query from the deployed model.
    Parameters:
    - model_id: <username>/<modelname>
    - query: The query to search for.
    - token: Authorization token from login
    """
    # Define the URL for querying the deployed model
    headers = {"Authorization": f"Bearer {token}"}
    query_url = f"{BASE_URL}/{model_id}/predict"
    # Set up the query parameters
    base_params = {"query": query, "top_k": 5}
    # Make a POST request to query the model
    response = requests.post(
        query_url,
        json=base_params,
        headers=headers,
    )
    # Check if the query was successful; if not, raise an exception
    if response.status_code != 200:
        raise Exception(f"Query failed: {response.status_code}, {response.text}")
    return response.json()["data"]["predicted_classes"]

In [ ]:
def token_classifier_predict(model_id: str, query: str, token: str, top_k=1):
    """
    Predicts the NER tags for a given query.
    Parameters:
    - model_id: model ID as returned by create_token_classifier. You can also find the model ID in the list returned by list_models.
    - query: The passage to predict the NER tags for.
    - token: Authorization token from login
    - top_k: The number of tags predicted for each token.
    Returns a dictionary in this format:
    {
        "text": "The text that was passed in",
        "predicted_tags": [
            ["TOP_TAG_FOR_FIRST_TOKEN", "SCORE_FOR_TOP_TAG_FOR_FIRST_TOKEN", ...],
            ["TOP_TAG_FOR_SECOND_TOKEN", "SCORE_FOR_TOP_TAG_FOR_SECOND_TOKEN", ...],
            ...
            ["TOP_TAG_FOR_LAST_TOKEN", "SCORE_FOR_TOP_TAG_FOR_LAST_TOKEN", ...]
        ]
    }
    The number of tags predicted for each token is specified in the top_k parameter.
    """
    headers = {"Authorization": f"Bearer {token}"}
    query_url = f"{BASE_URL}/{model_id}/predict"
    base_params = {"query": query, "top_k": top_k}
    response = requests.post(
        query_url,
        json=base_params,
        headers=headers,
    )
    # Check if the query was successful; if not, raise an exception
    if response.status_code != 200:
        raise Exception(f"Query failed: {response.status_code}, {response.text}")
    return response.json()["data"]

def obfuscate_pii(token_classifier_model_id: str, text_chunks: List[str], auth_token: str):
    """
    Obfuscates PII in the references using the NER model by replacing them with placeholders.
    Parameters:
    - token_classifier_model_id: model ID as returned by create_token_classifier. You can also find the model ID in the list returned by list_models.
    - text_chunks: A list of strings containing the text to obfuscate.
    - auth_token: Authorization token from login
    Returns a tuple containing:
    - A list of strings containing the obfuscated PII information.
    - A dictionary containing the mapping of obfuscated tokens to original tokens.
    """
    token_to_tag = {}
    token_counts = {}
    for text in text_chunks:
        text = " ".join(text.split())
        predicted_tags = token_classifier_predict(auth_token, token_classifier_model_id, text)
        predicted_tags = predicted_tags["predicted_tags"]
        for i, token in enumerate(text.split()):
            tag = predicted_tags[i][0]
            if tag != "O":
                if token not in token_to_tag:
                    tg = f"<{tag}>"
                    token_to_tag[token] = tg
    token_counts = {v: 0 for k, v in token_to_tag.items()}
    inverse_map = {}
    for k, v in token_to_tag.items():
        new_tag = v[:-1] + f"_{token_counts[v]}>"
        inverse_map[new_tag] = k
        token_to_tag[k] = new_tag
        token_counts[v] += 1
    output_text = []
    for text in text_chunks:
        text = " ".join(text.split())
        redacted_text = [
            word if word not in token_to_tag else token_to_tag[word]
            for word in text.split()
        ]
        output_text.append(" ".join(redacted_text))
    return output_text, inverse_map

def restore_pii(text: str, tag_to_token: Dict[str, str]):
    """
    Restores the PII in the text by replacing the placeholders with the original tokens.
    Parameters:
    - text: A string containing the obfuscated PII information.
    - tag_to_token: A dictionary containing the mapping of obfuscated tokens to original tokens.

    Returns a string containing the restored PII information.
    """
    restored_text = []
    for word in text.split():
        word = strip_non_alphanumeric(word)
        if word in tag_to_token.keys():
            restored_text.append(tag_to_token[word])
        else:
            restored_text.append(word)
    return " ".join(restored_text)


def strip_non_alphanumeric(word):
    pattern = r"^[^a-zA-Z0-9_<>\s]+|[^a-zA-Z0-9_<>\s]+$"
    cleaned_string = re.sub(pattern, "", word)
    return cleaned_string

In [ ]:
def chat(model_id: str, user_input: str, token: str, session_id: str = None):
    """
    Sends a chat request to the /chat endpoint.
    Parameters:
    - user_input: The message or query from the user.
    - token: Authorization token from login.
    - session_id: (Optional) Session ID for maintaining conversation context.
    Returns:
    - Response from the chat API.
    """
    chat_url = f"{BASE_URL}/{model_id}/chat"
    headers = {"Authorization": f"Bearer {token}"}
    payload = {
        "user_input": user_input,
        "session_id": session_id
    }
    response = requests.post(chat_url, json=payload, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Chat request failed: {response.status_code}, {response.text}")
    return response.json()

In [ ]:
token = login("admin@thirdai.com", "password")

results = query_retrieval_model("d4c6f0f2-6de1-4afb-b0c8-73adc2093094", "how to upgrade ios version", token)

# results is a list of dictioaries with references
# results[0].keys()
# dict_keys(['id', 'text', 'context', 'source', 'metadata', 'source_id', 'score'])

token_tags = token_classifier_predict("1c86a014-f724-4c29-95e8-bac292adfba4", "my name is david and my phone number is 9728172948", token)
# {'query_text': 'my name is david and my phone number is 9728172948', 'tokens': ['my', 'name', 'is', 'david', 'and', 'my', 'phone', 'number', 'is', '9729991112'], 'predicted_tags': [['O'], ['O'], ['O'], ['NAME'], ['O'], ['O'], ['O'], ['O'], ['O'], ['PHONENUMBER']]} 

sentiment_result = query_sentiment_model("ac58ccaf-8031-433a-9ab5-db20af83b978", "I hate this call", token)

sentiment_map = {"0": "negative", "1": "neutral", "2": "positive"}
sentiment = sentiment_map[sentiment_result[0][0]]
score = sentiment_result[0][1]

In [ ]:
token = login("admin@thirdai.com", "password")

sentiment_map = {"0": "negative", "1": "neutral", "2": "positive"}

session_id = str(uuid.uuid4())

while True:
    user_input = input("Enter your query: ")
    token_tags_result = token_classifier_predict("1c86a014-f724-4c29-95e8-bac292adfba4", user_input, token)
    tokens = token_tags_result['tokens']
    predicted_tags = token_tags_result['predicted_tags']
    sensitive_tokens = [tokens[i] for i in range(len(tokens)) if predicted_tags[i][0] != "O"]
    if sensitive_tokens:
        print("Sensitive Tokens Detected: ", sensitive_tokens)
    else:
        print("No Sensitive Tokens Detected.")
    sentiment_result = query_sentiment_model("ac58ccaf-8031-433a-9ab5-db20af83b978", user_input, token)
    sentiment = sentiment_map[sentiment_result[0][0]]
    score = sentiment_result[0][1]
    print(f"Sentiment: {sentiment} (score: {score})")
    retrieval_results = query_retrieval_model("d4c6f0f2-6de1-4afb-b0c8-73adc2093094", user_input, token)
    reference_text = "\n".join([result['text'] for result in retrieval_results])
    # print("Reference Text:\n", reference_text)
    prompt = f""
    response = chat("f7212483-e610-4d0f-b7b7-353cf307ed06", "how to upgrade ios version", token, session_id)